In [1]:
import openai
import os
import argparse
import requests
import json
from dotenv import load_dotenv

In [2]:
load_dotenv(".env")

True

In [3]:
MODEL_LIST = {
    "DeepSeek V3": {"model": "deepseek/deepseek-chat", 
                    "api_key": os.getenv("DEEPSEEK_API_KEY")},
    "QWEN 72B":{"model": "qwen/qwen-2.5-72b-instruct", 
                "api_key": os.getenv("QWEN_API_KEY")},
    "Llama 70B": {"model": "meta-llama/llama-3.3-70b-instruct",
                 "api_key": os.getenv("LLAMA_API_KEY")},
    "Claude 3.5 Haiku": {"model": "anthropic/claude-3.5-haiku", 
                        "api_key": os.getenv("CLAUDE_API_KEY")},
}

In [5]:
topic_list = [
    {
        "category": "Daily_Life",
        "prompts": [
            "Discuss your morning routine with a friend.",
            "Plan a weekend picnic with your family.",
            "Explain how you prepare your favorite meal to a new roommate."
        ]
    },
    {
        "category": "Shopping",
        "prompts": [
            "Ask a shop assistant for help finding a product.",
            "Negotiate a price for an item at a local market.",
            "Discuss a product return policy with a store employee."
        ]
    },
    {
        "category": "Travel_and_Transportation",
        "prompts": [
            "Describe your recent trip to a new city to a friend.",
            "Ask for directions to a nearby hotel in an unfamiliar town.",
            "Plan a group vacation, deciding on the destination and activities."
        ]
    },
    {
        "category": "Education",
        "prompts": [
            "Discuss an upcoming exam with a classmate.",
            "Explain a challenging topic to a peer in your study group.",
            "Ask your teacher for clarification on a homework assignment."
        ]
    },
    {
        "category": "Workplace",
        "prompts": [
            "Discuss a project deadline with your manager.",
            "Role-play a job interview, answering common questions.",
            "Explain a task to a new coworker."
        ]
    },
    {
        "category": "Health_and_Wellness",
        "prompts": [
            "Describe symptoms to a doctor during a visit.",
            "Give advice to a friend about staying healthy.",
            "Discuss exercise routines and fitness goals with a trainer."
        ]
    },
    {
        "category": "Social_Interactions",
        "prompts": [
            "Introduce yourself to someone at a party.",
            "Plan a birthday surprise for a friend with your group.",
            "Role-play a phone conversation with a distant relative."
        ]
    },
    {
        "category": "Problem_Solving",
        "prompts": [
            "Resolve a misunderstanding with a friend.",
            "Request assistance from customer service for a technical issue.",
            "Apologize for a mistake and offer a solution."
        ]
    },
    {
        "category": "Entertainment",
        "prompts": [
            "Discuss your favorite movies and TV shows with a friend.",
            "Plan an evening out, deciding between restaurants and activities.",
            "Recommend a book to someone and explain why they should read it."
        ]
    },
    {
        "category": "Cultural_Exchange",
        "prompts": [
            "Explain a tradition or festival from your country to a foreigner.",
            "Discuss cultural differences in dining etiquette with a friend.",
            "Ask a coworker about their favorite holiday in their country."
        ]
    }
]

new_topic_list = [
    {
        "category": "Daily Life",
        "prompts": [
        "Discuss your morning routine with a friend.",
        "Plan a weekend picnic with your family.",
        "Explain how you prepare your favorite meal to a new roommate."
        ]
    },
    {
        "category": "Shopping",
        "prompts": [
        "Ask a shop assistant for help finding a product.",
        "Negotiate a price for an item at a local market.",
        "Discuss a product return policy with a store employee."
        ]
    },
    {
        "category": "Travel and Transportation",
        "prompts": [
        "Describe your recent trip to a new city to a friend.",
        "Ask for directions to a nearby hotel in an unfamiliar town.",
        "Plan a group vacation, deciding on the destination and activities."
        ]
    },
    {
        "category": "Education",
        "prompts": [
        "Discuss an upcoming exam with a classmate.",
        "Explain a challenging topic to a peer in your study group.",
        "Ask your teacher for clarification on a homework assignment."
        ]
    },
    {
        "category": "Workplace",
        "prompts": [
        "Discuss a project deadline with your manager.",
        "Role-play a job interview, answering common questions.",
        "Explain a task to a new coworker."
        ]
    },
    {
        "category": "Health and Wellness",
        "prompts": [
        "Describe symptoms to a doctor during a visit.",
        "Give advice to a friend about staying healthy.",
        "Discuss exercise routines and fitness goals with a trainer."
        ]
    },
    {
        "category": "Social Interactions",
        "prompts": [
        "Introduce yourself to someone at a party.",
        "Plan a birthday surprise for a friend with your group.",
        "Role-play a phone conversation with a distant relative."
        ]
    },
    {
        "category": "Problem-Solving",
        "prompts": [
        "Resolve a misunderstanding with a friend.",
        "Request assistance from customer service for a technical issue.",
        "Apologize for a mistake and offer a solution."
        ]
    },
    {
        "category": "Entertainment",
        "prompts": [
        "Discuss your favorite movies and TV shows with a friend.",
        "Plan an evening out, deciding between restaurants and activities.",
        "Recommend a book to someone and explain why they should read it."
        ]
    },
    {
        "category": "Cultural Exchange",
        "prompts": [
        "Explain a tradition or festival from your country to a foreigner.",
        "Discuss cultural differences in dining etiquette with a friend.",
        "Ask a coworker about their favorite holiday in their country."
        ]
    },
            {
        "category": "Daily Conversations",
        "prompts": [
        "Talk about your favorite hobby and why you enjoy it.",
        "Discuss your plans for the weekend with a friend.",
        "Describe a recent memorable event and how it made you feel."
        ]
    },
    {
        "category": "Travel and Exploration",
        "prompts": [
        "Plan a trip to a new city with a group of friends.",
        "Describe your favorite place to visit and why it’s special.",
        "Ask for directions to a landmark in a new town."
        ]
    },
    {
        "category": "Workplace Interactions",
        "prompts": [
        "Discuss project deadlines and tasks with a coworker.",
        "Explain a process or task to a new team member.",
        "Plan a team meeting and decide on the agenda."
        ]
    },
    {
        "category": "Educational Settings",
        "prompts": [
        "Ask a teacher for clarification on an assignment.",
        "Discuss an upcoming test with a classmate and share study tips.",
        "Explain a challenging topic to a study group member."
        ]
    },
    {
        "category": "Shopping and Services",
        "prompts": [
        "Ask a shopkeeper for recommendations on a product.",
        "Discuss the features of an item before deciding to buy it.",
        "Explain an issue with a recent purchase to customer service."
        ]
    },
    {
        "category": "Health and Wellness",
        "prompts": [
        "Describe symptoms to a doctor during a consultation.",
        "Discuss exercise routines and goals with a fitness trainer.",
        "Give advice to a friend on staying healthy and active."
        ]
    },
    {
        "category": "Social Events and Gatherings",
        "prompts": [
        "Introduce yourself to someone new at a party.",
        "Plan a surprise party for a friend with a group.",
        "Talk about shared interests with someone you just met."
        ]
    },
    {
        "category": "Problem-Solving and Conflict Resolution",
        "prompts": [
        "Resolve a misunderstanding with a friend through discussion.",
        "Discuss ways to fix a technical issue with a service provider.",
        "Plan steps to address a shared problem with a coworker."
        ]
    },
    {
        "category": "Cultural Exchange and Traditions",
        "prompts": [
        "Describe a tradition or festival from your culture to a friend.",
        "Discuss differences in dining etiquette with someone from another culture.",
        "Plan a cultural event to share your traditions with others."
        ]
    },
    {
        "category": "Personal Goals and Experiences",
        "prompts": [
        "Talk about your goals for the next year and how you plan to achieve them.",
        "Describe a skill you want to learn and why it’s important to you.",
        "Discuss a recent personal achievement and what it means to you."
        ]
    }
]

In [6]:
def read_instruction(instruction_path):
    with open(instruction_path, 'r', encoding='utf-8') as file:
        instruction = file.read()
    return instruction

def request(api_key, 
            model, 
            output_dir_path, 
            system_instruction, 
            assist_instruction, 
            topic_list, 
            repeat_id=0,
            count=0):  # parameters
    for i in range(len(topic_list)):
        for j in range(len(topic_list[i]["prompts"])):
            message = assist_instruction.format(text=topic_list[i]["prompts"][j])
            try:
                messages = [
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": message},
                ]
                response = requests.post(
                    url="https://openrouter.ai/api/v1/chat/completions",
                    headers={
                        f"Authorization": f"Bearer {api_key}",
                    },
                    data=json.dumps({
                        "model": f"{model}",
                        "messages": messages
                    })
                )

                if response.status_code in [200, 401]:
                    try:
                        response_data = response.json()    
                        ans_model = response_data['choices'][0]['message'] 
                    except:
                        print(f"Error, Method 1 Reponse Data: {response_data}")
                else:
                    print(f"Method 1 Reponse Data: {response_data}")
                    print(f"Error: Received status code {response.status_code}")

            except requests.exceptions.RequestException as e:
                messages[0] = {"role": "user", "content": system_instruction}
                try:
                    response = requests.post(
                        url="https://openrouter.ai/api/v1/chat/completions",
                        headers={
                            "Authorization": f"Bearer f{api_key}",
                        },
                        data=json.dumps({
                            "model": f"{model}",
                            "messages": messages
                        })
                    )
                    if response.status_code in [200, 401]:
                        try:
                            response_data = response.json()  
                            ans_model = response_data['choices'][0]['message']  
                        except:
                            print(f"Error, Method 2 Reponse Data: {response_data}")
                    else:
                        print("Method 2 Response Data: ", response_data)
                        print(f"Error: Received status code {response.status_code}")

                except requests.exceptions.RequestException as e:
                    print(f"Request failed after retry: {e}")

            
            # print(f"Answer: {ans_model}")
            # ans_model = ans_model[3:-3].strip()
            # ans_model = ans_model.lstrip('json').strip()   
            print(f"Conversation Count: {count}, Category: {topic_list[i]['category']}")
            count += 1
            file_save_path = os.path.join(output_dir_path, f"{topic_list[i]['category']}_{j+repeat_id*3}.txt")
            with open(file_save_path, "w") as file:
                file.write(ans_model["content"])
    return  count

In [ ]:
model_name = "QWEN 72B"
api_key = MODEL_LIST[model_name]["api_key"]
model = MODEL_LIST[model_name]["model"]

context_instruction_dir_path = "../lib/instructions/context_instructions"  
generation_instruction_dir_path = "../lib/instructions/generation_instructions" 
output_dir = f"../data/{model_name}_output/"

count = 0
for context_instruction, generation_instruction in zip(sorted(os.listdir(context_instruction_dir_path)), 
                                                       sorted(os.listdir(generation_instruction_dir_path))):
    language = context_instruction.strip(".txt")
    context_instruction_path = os.path.join(context_instruction_dir_path, context_instruction)
    generation_instruction_path = os.path.join(generation_instruction_dir_path, generation_instruction)
    system_instruction = read_instruction(context_instruction_path)
    instruction = read_instruction(generation_instruction_path)
    output_dir_path = output_dir + f"{language}_dialog"
    if not os.path.exists(output_dir_path):
        os.makedirs(output_dir_path)
    count = request(api_key, model, output_dir_path, system_instruction, instruction, new_topic_list, count)

qwen/qwen-2.5-72b-instruct
Conversation Count: 0, Category: Daily Life
Conversation Count: 1, Category: Daily Life
Conversation Count: 2, Category: Daily Life
Conversation Count: 3, Category: Shopping
Conversation Count: 4, Category: Shopping
Conversation Count: 5, Category: Shopping
Conversation Count: 6, Category: Travel and Transportation
Conversation Count: 7, Category: Travel and Transportation
Conversation Count: 8, Category: Travel and Transportation
Conversation Count: 9, Category: Education
Conversation Count: 10, Category: Education
Conversation Count: 11, Category: Education
Conversation Count: 12, Category: Workplace
Conversation Count: 13, Category: Workplace
Conversation Count: 14, Category: Workplace
Conversation Count: 15, Category: Health and Wellness
Conversation Count: 16, Category: Health and Wellness
Conversation Count: 17, Category: Health and Wellness
Conversation Count: 18, Category: Social Interactions
Conversation Count: 19, Category: Social Interactions
Conve